# Constraint Programming

Following [this article](https://mlabonne.github.io/blog/constraintprogramming/)

## Satisfiability with the 3 scouts problem


In [10]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()
solver = cp_model.CpSolver()

army = model.NewIntVar(1, 10000, 'army')

model.AddModuloEquality(0, army, 13)
model.AddModuloEquality(0, army, 19)
model.AddModuloEquality(0, army, 37)

status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('================= Solution =================')
    print(f'Solved in {solver.WallTime():.2f} milliseconds')
    print()
    print(f'🪖 Army = {solver.Value(army)}')
    print()
    print('Check solution:')
    print(f' - Constraint 1: {solver.Value(army)} % 13 = {solver.Value(army) % 13}')
    print(f' - Constraint 2: {solver.Value(army)} % 19 = {solver.Value(army) % 19}')
    print(f' - Constraint 3: {solver.Value(army)} % 37 = {solver.Value(army) % 37}')

else:
    print('The solver could not find a solution.')


================= Solution =================
Solved in 0.00 milliseconds

🪖 Army = 9139

Check solution:
 - Constraint 1: 9139 % 13 = 0
 - Constraint 2: 9139 % 19 = 0
 - Constraint 3: 9139 % 37 = 0


In [26]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()
solver = cp_model.CpSolver()

army = model.NewIntVar(1, 100000, 'army')

model.AddModuloEquality(0, army, 13)
model.AddModuloEquality(0, army, 19)
model.AddModuloEquality(0, army, 37)

class PrintSolutions(cp_model.CpSolverSolutionCallback):
    """Callback to print every solution"""

    def __init__(self, variable):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variable = variable

    def on_solution_callback(self):
        print(self.Value(self.__variable))


solution_printer = PrintSolutions(army)
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, solution_printer)


9139
18278
27417
36556
45695
54834
63973
73112
82251
91390


## Optimization and beer

In [30]:
model = cp_model.CpModel()
solver = cp_model.CpSolver()

capacity = 19
bread = model.NewIntVar(0, capacity, 'bread')
meat = model.NewIntVar(0, capacity, 'meat')
beer = model.NewIntVar(0, capacity, 'beer')

model.Add(1 * bread + 3 * meat + 7 * beer <= capacity)

model.Maximize(3 * bread + 10 * meat + 26 * beer)

status = solver.Solve(model)

# If an optimal solution has been found, print results
if status == cp_model.OPTIMAL:
    print('================= Solution =================')
    print(f'Solved in {solver.WallTime():.2f} milliseconds')
    print()
    print(f'Optimal value = {3*solver.Value(bread)+10*solver.Value(meat)+26*solver.Value(beer)} popularity')
    print('Food:')
    print(f' - 🥖Bread = {solver.Value(bread)}')
    print(f' - 🥩Meat  = {solver.Value(meat)}')
    print(f' - 🍺Beer  = {solver.Value(beer)}')
else:
    print('The solver could not find an optimal solution.')


================= Solution =================
Solved in 0.00 milliseconds

Optimal value = 68 popularity
Food:
 - 🥖Bread = 2
 - 🥩Meat  = 1
 - 🍺Beer  = 2


In [33]:
model = cp_model.CpModel()
solver = cp_model.CpSolver()

capacity = 19
bread = model.NewIntVar(0, capacity, 'bread')
meat = model.NewIntVar(0, capacity, 'meat')
beer = model.NewIntVar(0, capacity, 'beer')

model.Add(1 * bread + 3 * meat + 7 * beer <= capacity)

# model.Maximize(3 * bread + 10 * meat + 26 * beer)

class CountSolutions(cp_model.CpSolverSolutionCallback):
    """Count the number of solutions."""

    def __init__(self):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1

    def solution_count(self):
        return self.__solution_count

solution_printer = CountSolutions()

solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, solution_printer)
print(solution_printer.solution_count())

121
